# Retina x APOE — AutoMorph Pipeline

Este notebook executa o AutoMorph no Google Colab (GPU T4) para extrair métricas vasculares de imagens de fundo de olho.

**Pré-requisitos:**
1. Rodar `01_match_patients.py --execute` localmente
2. Rodar `02_prepare_images.py --execute` localmente
3. Zipar a pasta `automorph_input/` e fazer upload para o Colab

**Output:** CSV com métricas vasculares (fractal dimension, tortuosity, density, CRAE, CRVE, AVR)

## 1. Setup — Instalar AutoMorph e dependências

In [ ]:
# Verificar GPU
!nvidia-smi

In [ ]:
# Clonar AutoMorph
!git clone https://github.com/rmaphoh/AutoMorph.git
%cd AutoMorph

In [ ]:
# Instalar dependências
!pip install -r requirements.txt

In [ ]:
# Download dos modelos pré-treinados
!python setup.py

## 2. Upload das imagens

In [ ]:
# Opção A: Upload manual do zip
from google.colab import files
import zipfile
import os

print("Faça upload do arquivo automorph_input.zip")
uploaded = files.upload()

# Extrair
zip_name = list(uploaded.keys())[0]
with zipfile.ZipFile(zip_name, 'r') as z:
    z.extractall('/content/input_images')

# Verificar se as imagens estão numa subpasta
input_dir = '/content/input_images'
contents = os.listdir(input_dir)
if len(contents) == 1 and os.path.isdir(os.path.join(input_dir, contents[0])):
    input_dir = os.path.join(input_dir, contents[0])

imgs = [f for f in os.listdir(input_dir) if f.endswith('.jpg')]
print(f"Imagens encontradas: {len(imgs)}")
print(f"Diretório: {input_dir}")

In [ ]:
# Opção B: Google Drive (alternativa se o zip for muito grande)
# from google.colab import drive
# drive.mount('/content/drive')
# input_dir = '/content/drive/MyDrive/retina_apoe/automorph_input'
# imgs = [f for f in os.listdir(input_dir) if f.endswith('.jpg')]
# print(f"Imagens encontradas: {len(imgs)}")

In [ ]:
# Copiar resolution_information.csv para o input_dir se não existir
import csv

res_file = os.path.join(input_dir, 'resolution_information.csv')
if not os.path.exists(res_file):
    print("Gerando resolution_information.csv com default 11 μm/pixel...")
    with open(res_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['imgName', 'pixelSizeInMicrons'])
        for img in imgs:
            writer.writerow([img, 11.0])
    print(f"Gerado com {len(imgs)} entradas")
else:
    print("resolution_information.csv já existe")

## 3. Executar AutoMorph

O AutoMorph tem 4 módulos sequenciais:
1. **Image preprocessing** — Crop/resize
2. **Quality grading** — EfficientNet-B4
3. **Vessel segmentation** — Binary + Artery/Vein
4. **Morphology measurement** — Métricas quantitativas

In [ ]:
# Configurar paths
import yaml

# AutoMorph usa um config YAML
config_path = '/content/AutoMorph/config.yaml'

# Ler config existente
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Atualizar paths
config['input_path'] = input_dir
config['output_path'] = '/content/automorph_output'

# Salvar config
with open(config_path, 'w') as f:
    yaml.dump(config, f)

print("Config atualizado:")
print(yaml.dump(config, default_flow_style=False))

In [ ]:
# Se o config YAML não funcionar, executar diretamente
# AutoMorph pode ser chamado via CLI
import subprocess

output_dir = '/content/automorph_output'
os.makedirs(output_dir, exist_ok=True)

# Executar pipeline completo
!cd /content/AutoMorph && python main.py --input_path {input_dir} --output_path {output_dir}

In [ ]:
# Se o comando acima não funcionar, tentar a abordagem por módulos individuais:
# !cd /content/AutoMorph && python M1_Preprocess/preprocess.py --input_path {input_dir} --output_path {output_dir}
# !cd /content/AutoMorph && python M2_Grading/grading.py --input_path {output_dir}/M1_output --output_path {output_dir}
# !cd /content/AutoMorph && python M3_Segmentation/segmentation.py --input_path {output_dir}/M2_output --output_path {output_dir}
# !cd /content/AutoMorph && python M4_Measurement/measurement.py --input_path {output_dir}/M3_output --output_path {output_dir}

## 4. Verificar resultados

In [ ]:
import pandas as pd
import glob

# Encontrar CSVs de resultado
result_csvs = glob.glob(f'{output_dir}/**/*.csv', recursive=True)
print("Arquivos CSV encontrados:")
for csv_file in result_csvs:
    print(f"  {csv_file}")
    df = pd.read_csv(csv_file)
    print(f"    Colunas: {list(df.columns)}")
    print(f"    Linhas: {len(df)}")
    print()

In [ ]:
# Carregar o CSV principal de métricas
# O AutoMorph gera um CSV consolidado com todas as métricas
# O nome exato depende da versão, mas geralmente é algo como:
# - measurement_results.csv
# - morphology_measurement.csv

# Encontrar o CSV de métricas
metrics_csv = None
for csv_file in result_csvs:
    df = pd.read_csv(csv_file)
    # O CSV de métricas terá colunas como fractal_dimension, tortuosity, etc.
    cols_lower = [c.lower() for c in df.columns]
    if any('fractal' in c or 'tortu' in c or 'density' in c for c in cols_lower):
        metrics_csv = csv_file
        break

if metrics_csv:
    print(f"CSV de métricas: {metrics_csv}")
    df = pd.read_csv(metrics_csv)
    print(f"Shape: {df.shape}")
    print(f"Colunas: {list(df.columns)}")
    print()
    print(df.head())
else:
    print("CSV de métricas não encontrado automaticamente.")
    print("Verifique manualmente os CSVs listados acima.")

## 5. Download dos resultados

In [ ]:
# Zipar resultados
import shutil

shutil.make_archive('/content/automorph_results', 'zip', output_dir)
print(f"Arquivo: /content/automorph_results.zip")
print(f"Tamanho: {os.path.getsize('/content/automorph_results.zip') / 1024 / 1024:.1f} MB")

In [ ]:
# Download do zip
from google.colab import files
files.download('/content/automorph_results.zip')

In [ ]:
# Ou salvar no Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# shutil.copy('/content/automorph_results.zip', '/content/drive/MyDrive/retina_apoe/')
# print("Salvo no Google Drive")

## 6. Análise rápida (preview)

Análise preliminar antes de rodar o script completo `04_analyze.py` localmente.

In [ ]:
# Upload do image_manifest.csv e matched_patients.csv
print("Faça upload do image_manifest.csv e matched_patients.csv")
uploaded2 = files.upload()

In [ ]:
if metrics_csv:
    import matplotlib.pyplot as plt
    import numpy as np

    metrics = pd.read_csv(metrics_csv)
    manifest = pd.read_csv('image_manifest.csv')

    # Merge metrics com manifest pelo nome do arquivo
    # Identificar coluna de nome de arquivo no metrics
    img_col = [c for c in metrics.columns if 'image' in c.lower() or 'file' in c.lower() or 'name' in c.lower()]
    print(f"Coluna de imagem detectada: {img_col}")

    if img_col:
        merged = manifest.merge(metrics, left_on='image_file', right_on=img_col[0], how='inner')
        print(f"Merged: {len(merged)} registros")
        print(f"Colunas: {list(merged.columns)}")

        # Fractal dimension por genótipo
        fd_col = [c for c in merged.columns if 'fractal' in c.lower()]
        if fd_col:
            genotype_order = ['e2e2', 'e2e3', 'e3e3', 'e3e4', 'e4e4']
            data = merged[merged['genotype'].isin(genotype_order)]
            grouped = data.groupby('genotype')[fd_col[0]].agg(['mean', 'std', 'count'])
            grouped = grouped.reindex(genotype_order)
            print("\nFractal Dimension por genótipo:")
            print(grouped)
    else:
        print("Não foi possível identificar coluna de nome de arquivo nos resultados.")
        print("Ajuste manualmente o merge.")